In [9]:
import cv2
import numpy as np
import time

In [33]:

#declare variables 
protoFile = "hand/pose_deploy.prototxt"
weightsFile = "hand/pose_iter_102000.caffemodel"
nPoints = 22
POSE_PAIRS = [ [0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20] ]
threshold=0.2

#get frame
frame = cv2.imread("img/8.jpg")
frameHeight,frameWidth= frame.shape[:2]
print( frame.shape[:2])

#finding new size
aspect_ratio = frameWidth/frameHeight
inHeight = 368
inWidth = int(((aspect_ratio*inHeight)*8)//8)

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),(0, 0, 0), swapRB=False, crop=False)
net.setInput(inpBlob)

(728, 620)


In [34]:
t = time.time()
output = net.forward()
print("forward = {}".format(time.time() - t))

print(len(output))

forward = 3.2166409492492676
1


In [35]:
points = []
frameWidth, frameHeight=frameWidth, frameHeight
frameCopy=frame.copy()

for i in range(nPoints):
    # confidence map of corresponding body's part.
    probMap = output[0, i, :, :]
    probMap = cv2.resize(probMap, (frameWidth, frameHeight))

    # Find global maxima of the probMap.
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

    if prob > threshold :
        cv2.circle(frameCopy, (int(point[0]), int(point[1])), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

       # Add the point to the list if the probability is greater than the threshold
        points.append((int(point[0]), int(point[1])))
    else :
        points.append(None)

        
frameCopy=cv2.resize(frameCopy, (500, 600))
cv2.imshow('Output-Keypoints', frameCopy)

cv2.waitKey()
cv2.destroyAllWindows()
#cv2.imwrite('asds.jpg', frameCopy) 

In [29]:
frameCopy1=frame.copy()

# Draw Skeleton
for pair in POSE_PAIRS:
    partA = pair[0]
    partB = pair[1]

    if points[partA] and points[partB]:
        cv2.line(frameCopy1, points[partA], points[partB], (0, 255, 255), 2)
        cv2.circle(frameCopy1, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        
cv2.imshow('Output-final', frameCopy1)
cv2.waitKey()
cv2.destroyAllWindows()

